# Autodiff:  <br> Новый взгляд на вычисления 

(Alan Edelman, 2017)


  Когда я впервые услышал об автоматической дифференцировании, мне было легко представить, как это работает. Но я был неправ. Я представлял это себе как простое символьное дифференцирование, применяемое к коду. Я считал, что это похоже на работу Mathematica или Maple или даже просто автоматическое выполнение того, чему я научился в университете. 
  <img src="images/derfunc.gif" width="230">
  .... в любом случае, даже если это не так, то это должно быть конечным дифференцированием , как то, которое мы все когда-то проходили.
  
<img src="http://image.mathcaptain.com/cms/images/122/Diff%202.png" width="150">



## Вавилонский корень

Я хотел бы показать простой пример, вычисления sqrt(x), где для меня то, как работает autodiff, стало и математическим сюрпризом, и вычислительным чудом. Примером является вавилонский алгоритм, известный человеку тысячелетиями, для вычисления sqrt(x):  


 > Repeat $ t \leftarrow  (t+x/t) / 2 $ until $t$ converges to $\sqrt{x}$.
 
 Каждая итерация имеет одно сложение и два деления. Для иллюстрации достаточно 10 итераций.

In [ ]:
function Babylonian(x; N = 10) 
    t = (1+x)/2
    for i = 2:N; t=(t + x/t)/2  end    
    t
end  

In [ ]:
Babylonian(π), √π   # \pi + <tab> , and \sqrt + <tab> 

Проверьте, что это работает:

In [ ]:
x=2; Babylonian(x),√x  # Type \sqrt+<tab> to get the symbol

In [ ]:
using Plots
gr()

In [ ]:
i = 0:0.01:49

plot([x->Babylonian(x,N=i) for i=1:5],i,label=["Iteration $j" for i=1:1,j=1:5])

plot!(sqrt,i,c="black",label="sqrt",
      title = "Those Babylonians really knew how to √")

## ...а теперь производная, почти по волшебству

In [ ]:
struct D <: Number  # D is a function-derivative pair
    f::Tuple{Float64,Float64}
end

Производная от суммы: $(x+y)' = x' + y'$ <br>
и от частного: $(x/y)' = (yx'-xy') / y^2$

In [ ]:
import Base: +, /, convert, promote_rule
+(x::D, y::D) = D(x.f .+ y.f)
/(x::D, y::D) = D((x.f[1]/y.f[1], (y.f[1]*x.f[2] - x.f[1]*y.f[2])/y.f[1]^2))
convert(::Type{D}, x::Real) = D((x,zero(x)))
promote_rule(::Type{D}, ::Type{<:Number}) = D

In [ ]:
x=2; Babylonian(D((x,1))),(√x,.5/√x)

In [ ]:
i = .2:0.01:49
plot([x->Babylonian(D((x,1.0)),N=i).f[2] for i=1:5],i)
plot!(x->.5/√x,i,c="black",label="d(sqrt(x))/dx",
    title = " Babylonians Differentiated")

## Оно работает!

Как это работает? Мы объясним через минуту. Прямо сейчас поражаюсь, что это так. Обратите внимание, что мы не импортировали ни один пакет autodiff. Все это просто ванильная Джулия.

## Символьно

Мы еще не объяснили, как это работает, но может быть полезно понять, что приведенное ниже будет математически эквивалентно, но не с точки зрения вычислений. 

Обратите внимание, что вавилонский язык работает с символами SymPy.

In [ ]:
# using Pkg; Pkg.add("SymPy")
using SymPy

In [ ]:
symbols("x");

In [ ]:
x = symbols("x")
display("Iterations as a function of x")
for k = 1:5
 display( simplify(Babylonian(x,N=k)))
end

display("Derivatives as a function of x")
for k = 1:5
 display(simplify(diff(simplify(Babylonian(x,N=k)),x)))
end

## Как autodiff получает ответ?

Давайте вручную возьмем «производную» вавилонской итерации по x. А именно t′=dt/dx

In [ ]:
function dBabylonian(x; N = 10) 
    t = (1+x)/2
    t′ = 1/2
    for i = 1:N;  
        t = (t+x/t)/2; 
        t′= (t′+(t-x*t′)/t^2)/2; 
    end    
    t′
end  

In [ ]:
x = π; dBabylonian(x), .5/√x

Что сейчас произошло? Ответ: Мы создали итерацию вручную для $t'$, учитывая нашу итерацию для $t$. Затем мы запустили их вместе.

In [ ]:
Babylonian(D((x,1)))

Как это работает? Наш метод создал ту же производную итерацию, используя очень общие правила, которые устанавливаются один раз и не требуют написания от руки.

Важно :: Производная подставляется перед JIT-компилятором, и, таким образом, выполняется эффективный скомпилированный код.

## Дуальная цифровая запись

Вместо `D(a,b)` мы можем записать `a + b ϵ`, где ϵ удовлетворяет $ϵ^2=0$.  (Некоторые люди сразу вспоминают мнимые числа, где **i** вводится как $i^2=-1$.) 

Другие вспоминают инженеров, которые любят отбрасывать второй порядок $O(ϵ^2)$.

Здесь работают 4 правила

$ (a+b\epsilon) \pm (c+d\epsilon) = (a \pm c) + (b \pm d)\epsilon$

$ (a+b\epsilon) * (c+d\epsilon) = (ac) + (bc+ad)\epsilon$

$ (a+b\epsilon) / (c+d\epsilon) = (a/c) + (bc-ad)/d^2 \epsilon $


In [ ]:
Base.show(io::IO,x::D) = print(io,x.f[1]," + ",x.f[2]," ϵ")

In [ ]:
# Add the last two rules
import Base: -,*
-(x::D, y::D) = D(x.f .- y.f)
*(x::D, y::D) = D((x.f[1]*y.f[1], (x.f[2]*y.f[1] + x.f[1]*y.f[2])))

In [ ]:
ϵ  = D((0,1))

In [ ]:
@code_native(ϵ^2)

In [ ]:
ϵ * ϵ

In [ ]:
1/(1+ϵ)  #Точный аналог ряда:  1-ϵ+ϵ²-ϵ³-...

In [ ]:
(1+ϵ)^10 ## Обратите внимание, оно работает само по себе (мы не учили Джулию возведению в степень)!!

## Обобщение на произвольные корни

In [ ]:
function nthroot(x, n=2; t=1, N = 10) 
    for i = 1:N;   t += (x/t^(n-1)-t)/n; end   
    t
end  

In [ ]:
nthroot(2,3), ∛2 # кубический корень

In [ ]:
nthroot(7,12), 7^(1/12)

In [ ]:
x = 17.0
nthroot( x+ϵ,3), ∛x, 1/x^(2/3)/3

## Прямое дифференцирование

Теперь, когда вы это поняли, вы можете использовать официальный пакет

In [ ]:
# using Pkg; Pkg.add("ForwardDiff")
using ForwardDiff

In [ ]:
ForwardDiff.derivative(sqrt, 2)

In [ ]:
ForwardDiff.derivative(Babylonian, 2)

In [ ]:
@which ForwardDiff.derivative(sqrt, 2)

## Подробней о сходимости

In [ ]:
setprecision(3000)
round.(Float64.(log10.([Babylonian(BigFloat(2),N=k) for k=1:10] .- √BigFloat(2))); digits=3)

In [ ]:
struct D1{T} <: Number  # D is a function-derivative pair
    f::Tuple{T,T}
end

In [ ]:
z = D((2.0,1.0))
z1 = D1((BigFloat(2.0),BigFloat(1.0)))

In [ ]:
import Base: +, /, convert, promote_rule
+(x::D1, y::D1) = D1(x.f .+ y.f)
/(x::D1, y::D1) = D1((x.f[1]/y.f[1], (y.f[1]*x.f[2] - x.f[1]*y.f[2])/y.f[1]^2))
convert(::Type{D1{T}}, x::Real) where {T} = D1((convert(T, x), zero(T)))
promote_rule(::Type{D1{T}}, ::Type{S}) where {T,S<:Number} = D1{promote_type(T,S)}

In [ ]:
A = randn(3,3)

In [ ]:
x = randn(3)

In [ ]:
ForwardDiff.gradient(x->x'A*x,x)

In [ ]:
(A+A')*x

Вынос мозга.... Я сам почти ничего не понял